# HopSkipDrive Take Home Test

In [1]:
import pandas as pd
import numpy as np
from xgboost import XGBClassifier, XGBRegressor
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.metrics import (accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, mean_absolute_error, mean_squared_error, r2_score)
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import ElasticNet
pd.set_option('display.max_columns', 70)

## Table of Contents

- [HopSkipDrive Take Home Test](#hopskipdrive-take-home-test)  
- [Prerequisites](#prerequisites)  
- [Research questions](#research-questions)  
  - [To what extent can we optimize the boosts to get rides claimed earlier?](#to-what-extent-can-we-optimize-the-boosts-to-get-rides-claimed-earlier)  
  - [To what extent can we optimize boosts to reduce the average cost of rides?](#to-what-extent-can-we-optimize-boosts-to-reduce-the-average-cost-of-rides)  
  - [To what extent can we have both 1 and 2?](#to-what-extent-can-we-have-both-1-and-2)  
- [Documentation & write up](#documentation-write-up)  
  - [What are your main recommendations based on the data?](#what-are-your-main-recommendations-based-on-the-data)  
  - [What other data might you want or need to test assumptions or improve model performance?](#what-other-data-might-you-want-or-need-to-test-assumptions-or-improve-model-performance)  
  - [How confident are you that the assumptions you made will generate improved outcomes?](#how-confident-are-you-that-the-assumptions-you-made-will-generate-improved-outcomes)  
  - [Write up a short description of any toolkits or processes that you believe would improve the workflow, the process, or the model performance](#write-up-a-short-description-of-any-toolkits-or-processes-that-you-believe-would-improve-the-workflow-the-process-or-the-model-performance)  
  - [Assume that this model is performant. What steps would you take to deliver the predictions or results routinely and at scale?](#assume-that-this-model-is-performant-what-steps-would-you-take-to-deliver-the-predictions-or-results-routinely-and-at-scale)  
- [Data preprocessing for analysis](#data-preprocessing-for-analysis)  
  - [Load the Data](#load-the-data)  
  - [Clean the data](#clean-the-data)  
- [Exploratory Data Analysis](#exploratory-data-analysis)  
  - [What are the factors that contribute to HopSkipDrive decision to boost a trip?](#what-are-the-factors-that-contribute-to-hopskipdrive-decision-to-boost-a-trip)  
    - [Dependent variable](#dependent-variable)  
    - [Independent variables](#independent-variables)  
  - [What is the optimal price per trip and by how much boosted trips deviate from it?](#what-is-the-optimal-price-per-trip-and-by-how-much-boosted-trips-deviate-from-it)  
- [Modeling the percentage deviation from the optimal price](#modeling-the-percentage-deviation-from-the-optimal-price)  
  - [Dependent variable](#dependent-variable)  
  - [Independent variables](#independent-variables)  
  - [Elastic Net Regression](#elastic-net-regression)  
  - [XGBoost Regressor](#xgboost-regressor)  
  - [Final model](#final-model)  


## Prerequisites
* This notebook assumes there is a directory called `data/` at the same directory level as the notebook and a file called `boost_df.csv` inside that directory.

## Research questions

### To what extent can we optimize the boosts to get rides claimed earlier?

Drivers decide when to claim a trip. Their decision depends on several factors, but the trip fare is the most important. HopSkipDrive (HSD) cannot control some of the trip's characteristics but can control the base trip fare and the subsequent boosts. If the only objective were for HSD to get rides claimed earlier, the solution would be to find the optimal boost that minimizes the time between trips being created and those claimed, given some constraints such as budget. Let's call this function f(x). However, HSD not only cares about this single objective optimization because it also cares about reducing the average cost of rides, leading to this assignment's second research question.   

### To what extent can we optimize boosts to reduce the average cost of rides?

HSD controls the trip fares and boosts. If the only objective was for HSD to reduce the average cost of rides, the solution would be to find the optimal boost that minimizes the boost amount given some constraints. Let's call this function g(x). Similar to the first question, HSD cares not only about this single objective optimization but also about getting rides claimed earlier, which leads to the third research question. 


### To what extent can we have both 1 and 2?

In practice, HSD is interested in a bi-objective optimization: min(f(x), g(x)) given some constraints. There is a natural trade-off between f(x) and g(x), and the set of optimal solutions would help HSD decide which trade-off works better for its business objective. We could use operations research methods to answer the question directly to find optimal solutions where neither f(x) nor g(x) can be further optimized. However, my perspective on this assignment as a data scientist is not to calculate the set of optimal solutions. Instead, I will analyze the boost data to understand the factors that lead drivers to expect higher boost amounts.

## Documentation & write up

### What are your main recommendations based on the data?

The core of my proposal lies in personalization. HSD runs a unique marketplace where it guarantees to meet the demand side of the business. However, drivers decide when to claim a trip. There are segments of drivers with different levels of sensitivity to boost amounts. A machine learning model would allow HSD to find those segments and score them by their likelihood of accepting distinct amounts of boosts. HSD could use those scores in a production model to roll out boosts by segments, starting with the segments that are more likely to accept smaller amounts of boosts. However, the data provided for this assignment does not include driver-specific information. Therefore, I built a toy model of a modified version of my proposal for this assignment: an Elastic Net regression that models the percentage increase in base fare as the dependent variable among trips that received a boost to measure the factors that lead drivers to expect a higher boost amount. 

- A few recommendations based on the results of the model:
    - Month seasonality is strongly positively related to an expected increase in the base fare amount. I recommend offering minor boosts during the months when drivers' expectations for significant boosts are lower. I'm not saying what specific months because I engineered month seasonality with sine and cosine transformations and decided not to back-engineer them to actual months to have time to finish the assignment. 
    - Offer lower boosts in metro areas where the expectation for high boosts is lower, such as DFW or Greensboro. Similarly, markets like DMV and Sacramento expect higher boosts. 
    - Trips that start at peak hours expect higher boosts. When calculating the base fare of peak-hours trips, consider the trip will likely need a boost. 
    - Drivers like longer trips, so consider that drivers have a lower expectation for larger boosts when the trip is longer in distance and in time.

- General recommendations based on working with this data:
    - Invest in acquiring more drivers. From an economic perspective, a larger supply should drive prices down.
    - Educate the demand side with analytics about rides. For example, you can share with the demand side from DMV that rides take longer to be claimed in that area, but ultimately, x% of rides are claimed x days before the scheduled start date.

### What other data might you want or need to test assumptions or improve model performance?

* I would like to have access to riders' and drivers' data. If I had that data, I would have built a toy model to illustrate the proposal I described in the previous questions.
* I would also like to have as much data as possible regarding more years and trips. 
* I would also have wanted additional data on the system that decides when a ride will receive a boost. Does an algorithm power it, or is it based on some rules that individuals apply? 

### How confident are you that the assumptions you made will generate improved outcomes?

* I am not confident that my assumptions will generate improved outcomes because this is a toy model with limited data developed with a short time limit. With more time and data, I could probably create a model that could potentially improve outcomes. However, instead of focusing on developing an accurate model, my objective for this assignment was to 1) understand the problem, 2) outline a potential solution, 3) clean the code in a Python script to run it in a Docker container to emulate a production setting, 4) and have enough time to document the process and answer the questions. 
* Some of the trade-offs I had to make to finish the steps described above in the allocated time are the following: limited exploratory data analysis, dropping rows without detailed investigation, avoiding visualizations, minimal feature engineering, only trying two regression models, not cross-validating results, and developing a model as quickly as possible. 

### Write up a short description of any toolkits or processes that you believe would improve the workflow, the process, or the model performance

* Before improving model performance, I would ensure the performance I'm reporting (R-squared of 50%) is genuine by checking that the model is not violating any assumptions and that there is no target leakage in the features. 
* To improve model performance, I would conduct more exploratory analysis to find and engineer additional features, try different algorithms, and spend more time on hyperparameter tuning, among other steps related to model selection. 
* To improve the process, I would modularize the code for production. I organized the Python code (model_pipeline.py) in a single script for readability for this assignment. However, in actual production, I would further modularize the model pipeline. I would also migrate all data cleaning to SQL or Spark. I would also define the MLOps pipeline: versioning, monitoring, training cadence, and prediction storage, among other steps. 
* To improve the workflow, I would use MLFlow. I intended to use MLFlow for this assignment but ran out of time when building the Docker container. I decided to skip the step and only mention it here. 

### Assume that this model is performant. What steps would you take to deliver the predictions or results routinely and at scale?

I briefly mentioned this in the previous answer. I would create a data pipeline in SQL using dbt to automate the data cleaning and pre-processing steps as much as possible. I would also modularize the Python model pipeline to align with best practices in reproducibility and MLOps. I would use MLFlow (it even has an integration with SageMaker) for MLOps. I would also dockerize the model to prepare and hand it off to the DevOps team. All these processes vary across companies' needs and resources. In HSD's case, I would learn about the tech stack and find what works best for the teams. 

## Data preprocessing for analysis

### Load the Data

In [2]:
df = pd.read_csv("data/boost_df.csv")
df.head(5)

,trip_id,cumulative_boost_amount_cents,boost_timestamp,manual_boost,boost_ind,seq_boost_count,single_boost_amount_cents,trip_state,created_at,claimed_at,scheduled_starts_at,scheduled_ends_at,unclaimed_at,trip_completed_at,total_predicted_duration_mins,total_predicted_distance_miles,total_predicted_distance_miles_for_fare,dollars_paid_to_driver,origin_metro_area_name,commute_minutes,commute_distance,is_same_day_ride,trip_starts_during_peak_hours,ever_unclaimed
0,43,0.0,NaN,0,0,0,0.0,complete,2023-06-01 17:20:28,2024-04-29 02:39:17,2024-05-17 22:25:00,2024-05-17 23:34:26,NaN,2024-05-17 23:30:55,48.60,24.88,24.88,34.43,Orlando Metro,35.78,4.88,False,True,0
1,58,492.1,2024-05-29 14:10:23,1,1,1,492.1,complete,2023-06-05 15:37:22,2024-05-29 14:40:50,2024-05-29 15:29:10,2024-05-29 16:10:00,2024-05-29 14:06:11,2024-05-29 16:10:33,28.58,19.19,19.19,34.73,Greensboro Metro,29.92,8.90,False,True,1
2,58,877.1,2024-05-29 14:37:26,1,1,2,385.0,complete,2023-06-05 15:37:22,2024-05-29 14:40:50,2024-05-29 15:29:10,2024-05-29 16:10:00,2024-05-29 14:06:11,2024-05-29 16:10:33,28.58,19.19,19.19,34.73,Greensboro Metro,29.92,8.90,False,True,1
3,62,588.0,2024-06-11 14:10:06,1,1,1,588.0,complete,2023-06-05 15:37:22,2024-06-11 14:15:20,2024-06-11 15:01:05,2024-06-11 15:40:00,2024-06-11 14:06:10,2024-06-11 15:47:50,27.24,19.19,19.19,31.37,Greensboro Metro,14.50,2.04,False,True,1
4,72,0.0,NaN,0,0,0,0.0,complete,2023-06-05 15:37:43,2024-05-28 17:37:30,2024-05-28 21:45:00,2024-05-28 22:29:17,2024-05-28 07:46:48,2024-05-28 22:29:06,31.00,19.40,19.40,32.83,Greensboro Metro,32.41,20.50,False,True,1


### Clean the data

* I'll start by casting date columns to datetime to be able to manipulate the dates. 

In [3]:
date_columns = [
    "boost_timestamp", "created_at", "claimed_at",  "scheduled_starts_at",
    "scheduled_ends_at", "unclaimed_at", "trip_completed_at"]
for col in date_columns:
    df[col] = pd.to_datetime(df[col], format="ISO8601")

In [4]:
# There are 5 trips with null values for commute distance. The number is so 
# small that I will just drop those trips
df = df[df.trip_id.isin(df[df.commute_distance.notnull()].trip_id.values)]

In [5]:
# All trips were completed so this column adds no information, I will drop it
assert (df.trip_state == "complete").sum() == df.shape[0]
df = df.drop("trip_state", axis=1)

In [6]:
# All trips have been claimed
assert df.claimed_at.isnull().sum() == 0

* There are boost events where `single_boost_amount_cents` is 0 cents. To simplify the cleaning process, I will assume that boost events with `boost_ind == 1` and `single_boost_amount_cents == 0` are invalid, and I will drop them. With more time, I would have looked more at this decision, but I think it's fair since the number of trips dropped is not too large (~300). 

In [7]:
trips_boost_yes_amount_zero = df[(df.boost_ind == 1) & (df.single_boost_amount_cents == 0)].trip_id.values
df = df[~df.trip_id.isin(trips_boost_yes_amount_zero)]

* Drop the seven trips where the claimed date is greater than the trip scheduled start. It's a rare event that returns negative numbers when calculating days between claimed date and scheduled start date. I'll drop these trips to simplify the cleaning process.

In [8]:
df = df[df.scheduled_starts_at > df.claimed_at]

In [9]:
print(f"Total number of rows: {df.shape[0]:,}")
print(f"Total number of trips: {df.trip_id.nunique():,}")

Total number of rows: 21,031
Total number of trips: 17,235


## Exploratory Data Analysis

### What are the factors that contribute to HopSkipDrive decision to boost a trip? 

* This is not the assignment's research question, but I think it is important to analyze it before starting with the research questions. 
* The decision to boost a trip is not a natural pattern in the data but rather the result of a system already built (I assume) by HSD. Therefore, this first step in the analysis aims to understand the factors that are considered by that system to boost a trip. Intuitively, I would assume HSD boosts trips when the scheduled start date is closer and the trip continues unclaimed. 
* In modeling terms, the dependent variable is `boost_ind` and the independent variables are the attributes of a trip known before the trip was boosted or started. For example, I can use `commute_distance` because the distance from the driver's origin (assuming it is fixed, like their house) is known before the trip was boosted. However, I cannot use the count of times the trip was boosted because it would introduce target leakage. After all, only the boosted trips would have a count greater than zero.
* In this section, I briefly analyze the relationship between a few independent variables and their effect on HSD's decision to boost the trip. After that, I fit an XGBoost Classifier to quantify the contribution of each independent variable to the dependent variable. 

#### Dependent variable

* Around 43% of the trips were boosted and 57% were not boosted. 

In [10]:
cnt_trips = df.trip_id.nunique()
cnt_boost = df[df.boost_ind == 1].trip_id.nunique()
boosted_pct = (cnt_boost / cnt_trips) * 100
print(f"Total trips: {cnt_trips:,}; boosted trips: {cnt_boost:,}; percentage boosted: {boosted_pct:.2f}%")

Total trips: 17,235; boosted trips: 7,245; percentage boosted: 42.04%


#### Independent variables

* Considering that the data provided is denormalized at the `trip_id` level, there are columns with values repeated for each trip_id, such as `commute_distance`. At this step, I am not analyzing anything related to boosts, so I can aggregate at the `trip_id` level, taking the maximum of those columns because all their rows for a given `trip_id` have the same value.

In [11]:
cols_agg = ["boost_ind", "created_at", "claimed_at", "scheduled_starts_at",
            "scheduled_ends_at",  "trip_completed_at", "unclaimed_at", 
            "total_predicted_duration_mins", "total_predicted_distance_miles_for_fare",
            "dollars_paid_to_driver", "origin_metro_area_name", "commute_minutes",
            "commute_distance",  "is_same_day_ride",  "trip_starts_during_peak_hours", 
            "ever_unclaimed"]
g_agg = df.groupby("trip_id", as_index=False)[cols_agg].max()
g_agg.head()

,trip_id,boost_ind,created_at,claimed_at,scheduled_starts_at,scheduled_ends_at,trip_completed_at,unclaimed_at,total_predicted_duration_mins,total_predicted_distance_miles_for_fare,dollars_paid_to_driver,origin_metro_area_name,commute_minutes,commute_distance,is_same_day_ride,trip_starts_during_peak_hours,ever_unclaimed
0,43,0,2023-06-01 17:20:28,2024-04-29 02:39:17,2024-05-17 22:25:00,2024-05-17 23:34:26,2024-05-17 23:30:55,NaT,48.60,24.88,34.43,Orlando Metro,35.78,4.88,False,True,0
1,58,1,2023-06-05 15:37:22,2024-05-29 14:40:50,2024-05-29 15:29:10,2024-05-29 16:10:00,2024-05-29 16:10:33,2024-05-29 14:06:11,28.58,19.19,34.73,Greensboro Metro,29.92,8.90,False,True,1
2,62,1,2023-06-05 15:37:22,2024-06-11 14:15:20,2024-06-11 15:01:05,2024-06-11 15:40:00,2024-06-11 15:47:50,2024-06-11 14:06:10,27.24,19.19,31.37,Greensboro Metro,14.50,2.04,False,True,1
3,72,0,2023-06-05 15:37:43,2024-05-28 17:37:30,2024-05-28 21:45:00,2024-05-28 22:29:17,2024-05-28 22:29:06,2024-05-28 07:46:48,31.00,19.40,32.83,Greensboro Metro,32.41,20.50,False,True,1
4,74,1,2023-06-05 15:37:43,2024-06-10 21:02:18,2024-06-10 21:45:00,2024-06-10 22:29:17,2024-06-10 22:47:29,2024-06-10 20:50:13,31.00,19.40,37.38,Greensboro Metro,35.17,18.91,False,True,1


* The first set of features I'm interested in exploring is measuring the time between key dates: days between trip created and trip claimed time, days between trip created and trip scheduled start time, and days between trip claimed and trip scheduled time. 
* It looks like the three variables could considerably affect HSD's decision to boost the trip.
* From the difference in means grouped by the dependent variable, I think the story is that boosted trips take less time to be claimed and are closer to the scheduled start time than trips not boosted by a factor of 10 to 15%. Most importantly, trips that were boosted were claimed on average 0.06 days before the scheduled start time, while trips that were not boosted were claimed on average 6 days before the scheduled start time.
* Considering the magnitude of the difference in the days between the claimed trip and the scheduled time, I suspect this is the key factor for HSD to boost a trip. In other words, when the trip is closer to the scheduled time without being claimed, it is considerably more likely to be boosted by HSD.

In [12]:
g_agg["days_btw_created_and_claimed"] = (
    g_agg.claimed_at - g_agg.created_at) / np.timedelta64(1, 'h') / 24
g_agg["days_btw_created_and_schedstart"] = (
    g_agg.scheduled_starts_at - g_agg.created_at) / np.timedelta64(1, 'h') / 24
g_agg["days_btw_claimed_and_schedstart"] = (
    g_agg.scheduled_starts_at - g_agg.claimed_at) / np.timedelta64(1, 'h') / 24
time_cols = ["boost_ind", "days_btw_created_and_claimed", "days_btw_created_and_schedstart", 
             "days_btw_claimed_and_schedstart"]
g_agg[time_cols].groupby("boost_ind").describe()

days_btw_created_and_claimed                                  \
                                 count       mean        std       min   
boost_ind                                                                
0                               9990.0  93.915687  99.352381  0.000104   
1                               7245.0  81.808711  92.438768  0.000093   

                                                         \
                 25%        50%         75%         max   
boost_ind                                                 
0           7.988938  49.147604  175.127054  363.199884   
1          12.590787  36.184144  126.780590  375.237442   

          days_btw_created_and_schedstart                                    \
                                    count        mean         std       min   
boost_ind                                                                     
0                                  9990.0  100.598898  100.369247  0.001389   
1                                  7245.0   81.845330   92.439651  0.009734   

                                                         \
                 25%        50%         75%         max   
boost_ind                                                 
0          15.122245  55.979740  182.160839  367.053009   
1          12.630394  36.223333  126.815613  375.255058   

          days_btw_claimed_and_schedstart                                \
                                    count      mean       std       min   
boost_ind                                                                 
0                                  9990.0  6.683211  8.415536  0.000937   
1                                  7245.0  0.036619  0.008803  0.001076   

                                                     
                25%       50%        75%        max  
boost_ind                                            
0          0.545625  2.076013  10.670932  30.939884  
1          0.032292  0.035370   0.042245   0.285058

* Looking at the cross tabulation of the variables `ever_unclaimed` and `boost_ind`, it is clear that when `trip_id` is true for `ever_unclaimed`, it is considerably more likely to be boosted. That makes sense because HSD must have rules to decide when a `trip_id` needs to be boosted and being `ever_unclaimed` must be one of the main variables. 
* However, I wonder what factors make HSD decide a `trip_id` is `unclaimed_at` at a given timestamp. I thought it could be determined by a fixed number of days/hours before the trip scheduled date, but a glance at the distribution showed that was not the case.

In [13]:
pd.crosstab(g_agg.boost_ind, g_agg.ever_unclaimed, margins=True, margins_name="Total", normalize=True)

ever_unclaimed,0,1,Total
boost_ind,,,
0,0.392167,0.187467,0.579634
1,0.137221,0.283145,0.420366
Total,0.529388,0.470612,1.000000


* I am also curious if the dynamics to decide if a trip needs to be boosted change based on `origin_metro_area_name`.
* A quick look at the distribution of `boost_ind` across metro areas shows some interesting differences, such as DFW having most of the trips not boosted. 

In [14]:
print(f"There are {g_agg.origin_metro_area_name.nunique():,} unique metro areas")
pd.crosstab(g_agg.boost_ind, g_agg.origin_metro_area_name, 
            margins=True, margins_name="Total", normalize=True)

There are 27 unique metro areas


origin_metro_area_name,ABQ Metro,Austin Metro,Bridgeport Metro,Cleveland Metro,Columbia Metro,DFW,DMV,Greater Boston,Greater Seattle,Greensboro Metro,Hampton Roads,Honolulu Metro,KC Metro,Milwaukee Metro,NOLA Metro,Nashville Metro,OKC Metro,Omaha Metro,Orlando Metro,Philly Metro,Providence Metro,Richmond Metro,SD Metro,Sacramento Metro,Sarasota Metro,Tucson Metro,Worcester Metro,Total
boost_ind,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,0.000348,0.015086,0.001393,0.004932,0.000116,0.120395,0.003481,0.014563,0.005396,0.051755,0.015782,0.005396,0.007253,0.004352,0.000348,0.01027,0.016652,0.005048,0.147317,0.005744,0.000580,0.008993,0.003597,0.090165,0.001276,0.000696,0.038700,0.579634
1,0.000464,0.018683,0.001857,0.002611,0.000116,0.031738,0.017348,0.018625,0.007079,0.047752,0.019669,0.001973,0.005570,0.006847,0.000522,0.01201,0.014099,0.006730,0.119988,0.003133,0.000870,0.007253,0.005686,0.036263,0.001741,0.000290,0.031448,0.420366
Total,0.000812,0.033768,0.003249,0.007543,0.000232,0.152132,0.020830,0.033188,0.012475,0.099507,0.035451,0.007369,0.012823,0.011198,0.000870,0.02228,0.030751,0.011778,0.267305,0.008877,0.001451,0.016246,0.009283,0.126429,0.003017,0.000986,0.070148,1.000000


* To use `origin_metro_area_name` in the model, I will make a one-hot-encoding of the metro area. 

In [15]:
g_agg["origin_metro_area_name"] = g_agg.origin_metro_area_name.str.lower().str.replace(" ","")
g_agg = pd.get_dummies(g_agg, dtype=int, prefix="", prefix_sep="")
g_agg.head()

,trip_id,boost_ind,created_at,claimed_at,scheduled_starts_at,scheduled_ends_at,trip_completed_at,unclaimed_at,total_predicted_duration_mins,total_predicted_distance_miles_for_fare,dollars_paid_to_driver,commute_minutes,commute_distance,is_same_day_ride,trip_starts_during_peak_hours,ever_unclaimed,days_btw_created_and_claimed,days_btw_created_and_schedstart,days_btw_claimed_and_schedstart,abqmetro,austinmetro,bridgeportmetro,clevelandmetro,columbiametro,dfw,dmv,greaterboston,greaterseattle,greensborometro,hamptonroads,honolulumetro,kcmetro,milwaukeemetro,nashvillemetro,nolametro,okcmetro,omahametro,orlandometro,phillymetro,providencemetro,richmondmetro,sacramentometro,sarasotametro,sdmetro,tucsonmetro,worcestermetro
0,43,0,2023-06-01 17:20:28,2024-04-29 02:39:17,2024-05-17 22:25:00,2024-05-17 23:34:26,2024-05-17 23:30:55,NaT,48.60,24.88,34.43,35.78,4.88,False,True,0,332.388067,351.211481,18.823414,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
1,58,1,2023-06-05 15:37:22,2024-05-29 14:40:50,2024-05-29 15:29:10,2024-05-29 16:10:00,2024-05-29 16:10:33,2024-05-29 14:06:11,28.58,19.19,34.73,29.92,8.90,False,True,1,358.960741,358.994306,0.033565,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,62,1,2023-06-05 15:37:22,2024-06-11 14:15:20,2024-06-11 15:01:05,2024-06-11 15:40:00,2024-06-11 15:47:50,2024-06-11 14:06:10,27.24,19.19,31.37,14.50,2.04,False,True,1,371.943032,371.974803,0.031771,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,72,0,2023-06-05 15:37:43,2024-05-28 17:37:30,2024-05-28 21:45:00,2024-05-28 22:29:17,2024-05-28 22:29:06,2024-05-28 07:46:48,31.00,19.40,32.83,32.41,20.50,False,True,1,358.083183,358.255058,0.171875,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,74,1,2023-06-05 15:37:43,2024-06-10 21:02:18,2024-06-10 21:45:00,2024-06-10 22:29:17,2024-06-10 22:47:29,2024-06-10 20:50:13,31.00,19.40,37.38,35.17,18.91,False,True,1,371.225405,371.255058,0.029653,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


* The final step before fitting the model is select the final set of independent variables and the dependent variable.

In [16]:
ind_cols = [col for col in g_agg if "_at" not in col and col not in ["trip_id", "boost_ind"]]
X = g_agg[ind_cols]
y = g_agg['boost_ind']

* As mentioned, I will use an XGBoost Classifier because it automatically handles interactions, handles multicollinearity, and does not require scaling, so preprocessing is minimal. This is not necessarily the best choice because, given the nature of the data, I did not take the time to consider additional models. Instead, I wanted a quick model to understand the feature's influence on the dependent variable. 

In [17]:
# Split the data
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y)
# Fit the model
xgbc = XGBClassifier(eval_metric='logloss', random_state=99)
xgbc.fit(X_train, y_train)
# Predict and get probabilities
y_pred = xgbc.predict(X_test)
y_proba = xgbc.predict_proba(X_test)[:, 1]
# Get classification metrics
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
roc_auc = roc_auc_score(y_test, y_proba)

* The model's performance is overestimated and clearly shows fundamental feature problems, such as target leakage. For that reason and in the interest of time, I won't bother interpreting the classification metrics. However, the results help me further understand how HSD decides a trip needs to be boosted. 
* The model results show that one of the variables (days between claimed time and scheduled start time) drives most of the HSD decision to boost a trip. Running the model just with that independent variable yields virtually the same results. 
* This first analysis helped me understand that the key driver for HSD to boost a trip is that the trip is coming up and has not been claimed. This was intuitive, but I wanted to see it in the data.

In [18]:
# Print performance metrics
print(f"Accuracy: {accuracy:.2f}")
print(f"Precision: {precision:.2f}")
print(f"Recall: {recall:.2f}")
print(f"F1-score: {f1:.2f}")
print(f"ROC AUC: {roc_auc:.2f}")
# Check feature importance
feat_imp = xgbc.feature_importances_
pd.DataFrame(
    {'Feature': X.columns, 'Importance': feat_imp}
).sort_values(by='Importance', ascending=False).reset_index(drop=True).head(10).T

Accuracy: 0.98
Precision: 0.96
Recall: 0.99
F1-score: 0.97
ROC AUC: 0.99


,0,1,2,3,4,5,6,7,8,9
Feature,days_btw_claimed_and_schedstart,dfw,ever_unclaimed,worcestermetro,greaterboston,greensborometro,clevelandmetro,days_btw_created_and_claimed,dollars_paid_to_driver,total_predicted_distance_miles_for_fare
Importance,0.696456,0.028175,0.020267,0.018644,0.01705,0.013482,0.012899,0.012578,0.012338,0.012


### What is the optimal price per trip and by how much boosted trips deviate from it?

* HSD guarantees the rides, so once they are scheduled, HSD needs to make sure they are claimed as soon as possible at the lowest price possible.
* I assume HSD has a pricing system that outputs the optimal pricing for each `trip_id`. Therefore, I will assume that if a trip was claimed without a boost, the dollar amount paid to the driver is the optimal price per trip. I will call that fare the `base_fare`. For trips that received a boost, `base_fare` would be `dollars_paid_to_driver` minus the sum of all the boosts that trip received. This assumes that `dollars_paid_to_driver` is the total final amount of dollars paid to a driver, including any boosts. 
* I will also define a column called `boost_prop_base_fare` to represent the percentage of the optimal price that the trip had to be boosted to claim. 

In [20]:
g_price = df.groupby(["trip_id", "boost_ind"], as_index=False).agg(
    dollars_paid_to_driver=("dollars_paid_to_driver", "max"), 
    total_boost_dollars=("cumulative_boost_amount_cents", lambda x: x.max() / 100)
)
def assign_base_fare(row):
    return row["dollars_paid_to_driver"] if row["boost_ind"] == 0 else row["dollars_paid_to_driver"] - row["total_boost_dollars"]
g_price["base_fare"] = g_price.apply(lambda x: assign_base_fare(x), axis=1)
g_price["boost_pct_base_fare"] = (g_price.total_boost_dollars / g_price.base_fare).mul(100).round(2)
g_price

,trip_id,boost_ind,dollars_paid_to_driver,total_boost_dollars,base_fare,boost_pct_base_fare
0,43,0,34.43,0.000,34.430,0.00
1,58,1,34.73,8.771,25.959,33.79
2,62,1,31.37,5.880,25.490,23.07
3,72,0,32.83,0.000,32.830,0.00
4,74,1,37.38,8.680,28.700,30.24
...,...,...,...,...,...,...
17230,383327,1,12.04,2.240,9.800,22.86
17231,383338,1,13.86,2.660,11.200,23.75
17232,383342,1,19.47,1.400,18.070,7.75
17233,383346,1,18.37,1.400,16.970,8.25


* Let's look at the distribution of percentage deviation from the optimal fare but only for trips that did not receive a boost. Trips that didn't receive a boost have a `boost_pct_base_fare` of zero.
* There is also one outlier for trip_id number 16426, where the calculation for `boost_pct_base_fare` is negative because total_boost_dollars is greater than `dollars_paid_to_driver`. I will drop that observation in the interest of time. Still, normally, I would explore doing something else, like truncating it to a very high distribution value, because it received an important boost.

In [21]:
g_price = g_price[g_price.boost_ind == 1]
g_price = g_price[g_price.boost_pct_base_fare >= 0 ]

* Clearly, the trips that do get a boost deviate considerably from their optimal price. Using the expected value, HSD would expect that any trip that needs a boost will require an increase of 89% of the original optimal base fare.

In [22]:
g_price.boost_pct_base_fare.describe(percentiles=[.1, .2, .3, .4, .5, .6, .7, .8, .9, .92, .94, .96, .98, .99, .999]).to_frame().T.round(2)

,count,mean,std,min,10%,20%,30%,40%,50%,60%,70%,80%,90%,92%,94%,96%,98%,99%,99.9%,max
boost_pct_base_fare,7244.0,89.08,74.44,0.55,10.53,17.45,31.15,49.03,71.69,95.57,122.75,153.98,198.53,208.69,222.0,243.09,270.17,296.92,351.39,450.0


* By fitting a linear model, we could further condition the expected value to see if the expectation changes based on covariate values. That would give us additional information about the covariates that increase or decrease the sensitivity to price changes. 
* I will build another toy model to explore this idea in the next section.
* However, at this point of the analysis, I think it is worth noting that ideally, I would like to measure the sensitivity of drivers to changes in the base fare to score drivers by likelihood of accepting the base fare. However, I don't have driver specific information. Driver data would be key to further developing this analysis, especially considering that how much boosts can be optimized to be claimed earlier depends on the drivers. If I had driver data, I would propose building a model scoring each potential driver based on the characteristics of the trip to see how likely they would be to accept the base fare. Then I would propose segmenting the drivers by the likelihood and, once HSD system decides a boost is needed, I would offer personalized boosts to drivers in small increments starting with the drivers that are more likely to accept the base fare and continuing with the drivers that are less likely to accept the slight boost. Then I would repeat with an additional small boost.
* For now, I'll continue with model deviation from the optimal price using trip specific information.

## Modeling the percentage deviation from the optimal price

### Dependent variable

* The unit of analysis is the trips flagged by the HSD system as needing a boost.
* The objective is to measure the deviation from the optimal changes conditional on the covariates by how much. 
* The dependent variable would be `boost_pct_base_fare`.

In [23]:
df_y = g_price.copy(deep=True)[["trip_id", "boost_pct_base_fare"]]
df_y.head(5)

,trip_id,boost_pct_base_fare
1,58,33.79
2,62,23.07
4,74,30.24
5,75,12.83
6,109,13.80


### Independent variables

* In the interest of time, I will reuse the independent variables used in the previous analysis section to get the independent variables for this model. I only reuse the variables from the earlier analysis of this notebook as part of the model development. The final model in this submission's file `model_pipeline.py` cleans out this process. 
* I won't use anything related to the amount of the boost because that's target leakage.
* For the assignment's request to do at least two feature engineering processes, I will use the metro area's one-hot encoding.

In [24]:
# Because of the dummies, it's easier to list the columns that I don't want
cols_not = [
    "boost_ind", "created_at", "claimed_at", "ever_unclaimed",
    "scheduled_starts_at", "scheduled_ends_at", "trip_completed_at",
    "unclaimed_at", "dollars_paid_to_driver"]
df_trip = g_agg.drop(cols_not, axis=1)
df_trip.head(5)

,trip_id,total_predicted_duration_mins,total_predicted_distance_miles_for_fare,commute_minutes,commute_distance,is_same_day_ride,trip_starts_during_peak_hours,days_btw_created_and_claimed,days_btw_created_and_schedstart,days_btw_claimed_and_schedstart,abqmetro,austinmetro,bridgeportmetro,clevelandmetro,columbiametro,dfw,dmv,greaterboston,greaterseattle,greensborometro,hamptonroads,honolulumetro,kcmetro,milwaukeemetro,nashvillemetro,nolametro,okcmetro,omahametro,orlandometro,phillymetro,providencemetro,richmondmetro,sacramentometro,sarasotametro,sdmetro,tucsonmetro,worcestermetro
0,43,48.60,24.88,35.78,4.88,False,True,332.388067,351.211481,18.823414,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
1,58,28.58,19.19,29.92,8.90,False,True,358.960741,358.994306,0.033565,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,62,27.24,19.19,14.50,2.04,False,True,371.943032,371.974803,0.031771,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,72,31.00,19.40,32.41,20.50,False,True,358.083183,358.255058,0.171875,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,74,31.00,19.40,35.17,18.91,False,True,371.225405,371.255058,0.029653,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


* During the panel, it was mentioned that HSD has a lot of seasonality. I think it would be valuable for the model to account for that. To simplify things, I will only use `scheduled_starts_at` to calculate seasonality features.
* For another example of feature engineering from the two requested in the assignment, I do cosine and sine transformations of month, day of the week, and hour of `scheduled_starts_at` to capture the cyclical nature of time and avoid the discontinuity introduced by an ordinal representation of dates and time. Note that I use both sine and cosine transformation for each feature to allow the feature to be more flexible and capture the seasonal effects.
* The problem with sine and cosine features is that they're harder to interpret in a regression context. 

In [25]:
df_sched = g_agg.copy(deep=True)[["trip_id", "scheduled_starts_at"]]
df_sched['sin_month'] = np.sin(2 * np.pi * df_sched['scheduled_starts_at'].dt.month / 12)
df_sched['cos_month'] = np.cos(2 * np.pi * df_sched['scheduled_starts_at'].dt.month / 12)
df_sched['sin_day_of_week'] = np.sin(2 * np.pi * df_sched['scheduled_starts_at'].dt.dayofweek / 7)
df_sched['cos_day_of_week'] = np.cos(2 * np.pi * df_sched['scheduled_starts_at'].dt.dayofweek / 7)
df_sched['sin_hour'] = np.sin(2 * np.pi * df_sched['scheduled_starts_at'].dt.hour / 24)
df_sched['cos_hour'] = np.cos(2 * np.pi * df_sched['scheduled_starts_at'].dt.hour / 24)
df_sched.head()

,trip_id,scheduled_starts_at,sin_month,cos_month,sin_day_of_week,cos_day_of_week,sin_hour,cos_hour
0,43,2024-05-17 22:25:00,5.000000e-01,-0.866025,-0.433884,-0.900969,-0.500000,0.866025
1,58,2024-05-29 15:29:10,5.000000e-01,-0.866025,0.974928,-0.222521,-0.707107,-0.707107
2,62,2024-06-11 15:01:05,1.224647e-16,-1.000000,0.781831,0.623490,-0.707107,-0.707107
3,72,2024-05-28 21:45:00,5.000000e-01,-0.866025,0.781831,0.623490,-0.707107,0.707107
4,74,2024-06-10 21:45:00,1.224647e-16,-1.000000,0.000000,1.000000,-0.707107,0.707107


* Now let's put all the independent variables together.

In [26]:
df_x = pd.merge(df_sched, df_trip, on="trip_id", how="inner", validate="1:1")
df_x.head()

,trip_id,scheduled_starts_at,sin_month,cos_month,sin_day_of_week,cos_day_of_week,sin_hour,cos_hour,total_predicted_duration_mins,total_predicted_distance_miles_for_fare,commute_minutes,commute_distance,is_same_day_ride,trip_starts_during_peak_hours,days_btw_created_and_claimed,days_btw_created_and_schedstart,days_btw_claimed_and_schedstart,abqmetro,austinmetro,bridgeportmetro,clevelandmetro,columbiametro,dfw,dmv,greaterboston,greaterseattle,greensborometro,hamptonroads,honolulumetro,kcmetro,milwaukeemetro,nashvillemetro,nolametro,okcmetro,omahametro,orlandometro,phillymetro,providencemetro,richmondmetro,sacramentometro,sarasotametro,sdmetro,tucsonmetro,worcestermetro
0,43,2024-05-17 22:25:00,5.000000e-01,-0.866025,-0.433884,-0.900969,-0.500000,0.866025,48.60,24.88,35.78,4.88,False,True,332.388067,351.211481,18.823414,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
1,58,2024-05-29 15:29:10,5.000000e-01,-0.866025,0.974928,-0.222521,-0.707107,-0.707107,28.58,19.19,29.92,8.90,False,True,358.960741,358.994306,0.033565,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,62,2024-06-11 15:01:05,1.224647e-16,-1.000000,0.781831,0.623490,-0.707107,-0.707107,27.24,19.19,14.50,2.04,False,True,371.943032,371.974803,0.031771,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,72,2024-05-28 21:45:00,5.000000e-01,-0.866025,0.781831,0.623490,-0.707107,0.707107,31.00,19.40,32.41,20.50,False,True,358.083183,358.255058,0.171875,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,74,2024-06-10 21:45:00,1.224647e-16,-1.000000,0.000000,1.000000,-0.707107,0.707107,31.00,19.40,35.17,18.91,False,True,371.225405,371.255058,0.029653,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


* And here is the full data ready for modeling. 

In [27]:
df_model = pd.merge(df_y, df_x, on="trip_id", how="inner", validate="1:1").drop(["scheduled_starts_at", "trip_id"], axis=1)
df_model.head()

,boost_pct_base_fare,sin_month,cos_month,sin_day_of_week,cos_day_of_week,sin_hour,cos_hour,total_predicted_duration_mins,total_predicted_distance_miles_for_fare,commute_minutes,commute_distance,is_same_day_ride,trip_starts_during_peak_hours,days_btw_created_and_claimed,days_btw_created_and_schedstart,days_btw_claimed_and_schedstart,abqmetro,austinmetro,bridgeportmetro,clevelandmetro,columbiametro,dfw,dmv,greaterboston,greaterseattle,greensborometro,hamptonroads,honolulumetro,kcmetro,milwaukeemetro,nashvillemetro,nolametro,okcmetro,omahametro,orlandometro,phillymetro,providencemetro,richmondmetro,sacramentometro,sarasotametro,sdmetro,tucsonmetro,worcestermetro
0,33.79,5.000000e-01,-0.866025,0.974928,-0.222521,-0.707107,-0.707107,28.58,19.19,29.92,8.90,False,True,358.960741,358.994306,0.033565,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,23.07,1.224647e-16,-1.000000,0.781831,0.623490,-0.707107,-0.707107,27.24,19.19,14.50,2.04,False,True,371.943032,371.974803,0.031771,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,30.24,1.224647e-16,-1.000000,0.000000,1.000000,-0.707107,0.707107,31.00,19.40,35.17,18.91,False,True,371.225405,371.255058,0.029653,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,12.83,1.224647e-16,-1.000000,-0.433884,-0.900969,-0.707107,0.707107,31.00,19.40,9.74,4.25,False,True,375.237442,375.255058,0.017616,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,13.80,5.000000e-01,-0.866025,0.974928,-0.222521,-0.707107,-0.707107,36.73,22.25,20.34,10.82,False,True,354.845463,354.870347,0.024884,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


* I will also split the data into a training and a test set.
* With more time, I would have also created a validation set to use only the test set one time after model selection. However, I'll do model selection with the test set to simplify the process. 

In [28]:
X = df_model.drop(columns=["boost_pct_base_fare"])
y = df_model["boost_pct_base_fare"]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=99)

* As a final step, I will scale the features with a standard scaler, especially because one of the model I'll try is a regression, which are sensitive to scaling issues. 

In [29]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

### Elastic Net Regression

* I will use an Elastic Net regression to start modeling.
* I like this model when I need to do something quick because it balances Lasso and Ridge regressions but still has the regression's interpretability. As a result, Elastic Net helps with feature selection and dealing with highly correlated features, which is the case in my data. 

In [30]:
elastic_net = ElasticNet(random_state=99)
param_dist = {
    "alpha": [.1, .2, .3, .4, .5, .6, .7, .8, .9],
    "l1_ratio": [.05, .15, .25, .35, .45, .55, .65, .75, .85, .95]
}
random_search = RandomizedSearchCV(
    elastic_net, param_distributions=param_dist, n_iter=10, scoring="r2", 
    cv=10, random_state=99, n_jobs=-1
)
random_search.fit(X_train_scaled, y_train)
elastic_net = random_search.best_estimator_

y_pred = elastic_net.predict(X_test_scaled)
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

* The model's performance looks acceptable with an R squared of 50%. It is a toy model, and some regression assumptions are likely being violated. Still, it's a good indicator to have an R squared of 50%, especially considering that the dependent variable seems to be, at least from my intuition, a complex variable to predict.
* The MAE seems high, indicating that the predictions are not precise.

In [31]:
print(f"Mean Absolute Error: {int(mae):,}")
print(f"Mean Squared Error: {int(mse):,}")
print(f"R Score: {r2:.2f}")

Mean Absolute Error: 40
Mean Squared Error: 2,703
R Score: 0.51


* One of the main advantages of using a regression like Elastic Net is the interpretability. As mentioned, many assumptions should be considered before running a regression, which I have skipped for the sake of time. However, I would spend more time making sure those assumptions are not violated at this step. 
* Normally, I would take a reasonable amount of time to interpret these results to ensure they make sense and align with business logic. However, to be able to continue working on the assignment, I'll briefly mention a high-level interpretation.
* The coefficients show a strong seasonality component explaining the positive deviation from the base fare. The coefficients show a very strong positive relation between the sine and cosine of the month. Similarly, there is a very strong negative relation with the cosine of hours. With more time, I would engineer back the transformations to actual hours to give a more intuitive explanation of seasonality and the dependent variable. I'll stop here at the moment to be able to continue.
* Another interesting coefficient is the negative relation that `total_predicted_ditance_miles_for_fare` has with the dependent variable. Drivers are more willing to accept longer trips without a boost.
* Another interesting result is the negative relationship of `total_predicted_duration_mins` and the dependent variable, suggesting that longer trips require fewer boosts because they're already attractive to drivers without the boost. 
* Finally, metro areas seem to have significant effects. For example, the drivers from DFW are more willing to accept lower boosts while the drivers from DMV tend to expect higher boosts. 

In [32]:
elastic_net_coeffs = pd.DataFrame({
    "Feature": X.columns,
    "Coefficient": elastic_net.coef_
})
elastic_net_coeffs.sort_values(by="Coefficient", ascending=False).set_index("Feature").T

Feature,sin_month,dmv,cos_month,sacramentometro,trip_starts_during_peak_hours,days_btw_claimed_and_schedstart,sarasotametro,clevelandmetro,richmondmetro,bridgeportmetro,cos_day_of_week,commute_distance,greaterseattle,okcmetro,days_btw_created_and_claimed,austinmetro,days_btw_created_and_schedstart,sin_hour,orlandometro,phillymetro,sin_day_of_week,milwaukeemetro,sdmetro,nolametro,tucsonmetro,abqmetro,columbiametro,providencemetro,commute_minutes,nashvillemetro,worcestermetro,hamptonroads,honolulumetro,kcmetro,greaterboston,omahametro,is_same_day_ride,cos_hour,total_predicted_duration_mins,greensborometro,dfw,total_predicted_distance_miles_for_fare
Coefficient,39.069581,16.093926,14.287279,9.873706,9.428259,9.136791,4.33341,3.726551,3.498734,3.41498,3.053114,2.581634,1.886808,1.190448,1.004795,0.869148,0.860789,0.492045,0.45934,0.291289,0.284216,0.062477,0.035909,-0.0,-0.0,-0.357562,-0.413857,-0.718632,-0.925954,-1.832734,-2.021938,-2.254483,-2.795389,-3.259201,-3.376267,-4.586749,-4.832293,-8.162534,-8.412902,-8.78086,-10.704176,-14.096357


### XGBoost Regressor

* I like fitting XGBoost models as a baseline and a quick solution when I need a model quickly. They require minimum preprocessing and perform great. That's why I decided also to fit an XGBoost Regressor and see what we can learn. 

In [33]:
xgbr = XGBRegressor(objective="reg:squarederror", random_state=99)
param_dist = {
    "max_depth": [3, 6, 9],
    "n_estimators": [50, 100, 200, 400, 500],
    "learning_rate": [.01, .1, .3, .4, .5],
    "min_child_weight": [.001, .01, .1, .3, .6, .9],
    "gamma": [.001, .01, .1, 1, 10],
    "colsample_bytree": [.3, .5, .7, .8, 1],
    "lambda": [1, 3, 6, 9, 12, 20, 30],
}

random_search = RandomizedSearchCV(
    xgbr, param_distributions=param_dist, n_iter=50, scoring="r2", 
    cv=10, random_state=99, n_jobs=-1
)
random_search.fit(X_train_scaled, y_train)
xgbr = random_search.best_estimator_

y_pred = xgbr.predict(X_test_scaled)
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

* The XGBoost Regressor outperforms the Elastic Net with an R squared 71% and a MAE of 29.
* I expected these results because this algorithm tends to perform very well and introduce interactions and other features as opposed to the Elastic Net where it is necessary to define them in the feature engineering process.

In [34]:
print(f"Mean Absolute Error: {mae}")
print(f"Mean Squared Error: {mse}")
print(f"R Score: {r2}")

Mean Absolute Error: 27.2583467818231
Mean Squared Error: 1522.9846995526882
R Score: 0.7243108625007344


* The only problem with the XGBoost Regressor compared to the Elastic Net is that it is more of a black box model. We get the feature importance, but they cannot be interpreted as coefficients from regression. In any case, it's reassuring that the XGBoost Regressor finds similar features impactful in explaining the dependent variable.

In [35]:
# Get feature importances
xgbr_feat_importance = pd.DataFrame({
    "Feature": X.columns,
    "Importance": xgbr.feature_importances_
})

# Print feature importance
xgbr_feat_importance.sort_values(by="Importance", ascending=False).set_index("Feature").T

Feature,dmv,sacramentometro,sin_month,dfw,trip_starts_during_peak_hours,clevelandmetro,sarasotametro,greensborometro,omahametro,is_same_day_ride,richmondmetro,greaterseattle,bridgeportmetro,cos_hour,total_predicted_distance_miles_for_fare,okcmetro,nashvillemetro,orlandometro,milwaukeemetro,cos_month,days_btw_claimed_and_schedstart,total_predicted_duration_mins,worcestermetro,phillymetro,austinmetro,greaterboston,kcmetro,honolulumetro,days_btw_created_and_schedstart,sin_hour,days_btw_created_and_claimed,hamptonroads,sdmetro,nolametro,commute_distance,cos_day_of_week,commute_minutes,sin_day_of_week,abqmetro,tucsonmetro,providencemetro,columbiametro
Importance,0.261393,0.106757,0.10264,0.056939,0.038362,0.033349,0.032938,0.027074,0.026976,0.026039,0.023063,0.019435,0.016886,0.015235,0.014514,0.014344,0.013568,0.013484,0.012873,0.011668,0.010827,0.010471,0.010205,0.009864,0.008414,0.008253,0.008103,0.007884,0.007349,0.007247,0.006165,0.005535,0.005378,0.004956,0.004136,0.003556,0.003216,0.002503,0.002357,0.002329,0.002069,0.001646


### Final model

- Considering the above model results, I will use the Elastic Net to fit my final model in the python script called `model_pipeline.py`.
- I decided that because I think gaining insights through interpretability is more important than achieving a better R squared using the XGBoost Regressor for this particular model.